In [ ]:
https://www.youtube.com/watch?v=oGgE180oaTs&list=PLBTZqjSKn0IeKBQDjLmzisazhqQy4iGkb&index=5

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("PySparkTables").getOrCreate()

# Define schema for customer_orders table
customer_orders_schema = StructType([
    StructField("product_id", IntegerType(), True),
    StructField("sales", IntegerType(), True)
])

# Create DataFrame with initial data
customer_orders_data = [
    (1, 2000),
    (2, 2500),
    (3, 2100),
    (4, 2000),
    (5, 2200),
    (6, 2700),
    (7, 3000),
    (8,1000),
    (9, 3000)
]

# Create DataFrame
orders_df = spark.createDataFrame(customer_orders_data, schema=customer_orders_schema)

# Convert order_date column to DateType

# Create a temporary view for SQL queries
orders_df.createOrReplaceTempView("orders")

print("customer_orders table and view created successfully.")


customer_orders table and view created successfully.


In [22]:
spark.sql("""
    select product_id from
    (select 
        product_id,
        sum(sales) over(order by sales rows between unbounded preceding and current row) as running_sales,
        0.8 * sum(sales) over() as total_sales
    from orders)
    where running_sales < total_sales
""").show()

+----------+
|product_id|
+----------+
|         8|
|         1|
|         4|
|         3|
|         5|
|         2|
|         6|
+----------+



In [35]:
window_spec = Window.orderBy("sales").rowsBetween(Window.unboundedPreceding, Window.currentRow)
window_spec1 = Window.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

orders_df.select(
    "product_id",
    sum(col("sales")).over(window_spec).alias("running_sales"),
    (lit(0.8) * sum(col("sales")).over(window_spec1)).alias("total_sales")
).show()

+----------+-------------+-----------+
|product_id|running_sales|total_sales|
+----------+-------------+-----------+
|         8|         1000|    16400.0|
|         1|         3000|    16400.0|
|         4|         5000|    16400.0|
|         3|         7100|    16400.0|
|         5|         9300|    16400.0|
|         2|        11800|    16400.0|
|         6|        14500|    16400.0|
|         7|        17500|    16400.0|
|         9|        20500|    16400.0|
+----------+-------------+-----------+

